In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
#import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_absolute_error
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [71]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #tf.set_random_seed(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

In [72]:
seed_everything(seed=1234)

In [73]:
train = pd.read_csv('data/train.csv')
pred_train = pd.read_csv('data/train_pseudo.csv')
pred_test = pd.read_csv('data/subs/blends/best_subs2.csv')
sub = pd.read_csv('data/sample_submission.csv')

In [74]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [75]:
pred_train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [76]:
pred_test.head()

,id,scalar_coupling_constant
0,4658147,9.850896
1,4658148,177.447117
2,4658149,2.044971
3,4658150,175.821144
4,4658151,10.297093


# Model

```
x_train = predictions of training
y_train = real training constant
x_test = predictions/submission
y_test = improved predictions ¿?
```


In [77]:
x_train = pred_train.scalar_coupling_constant.values.reshape(-1, 1)
y_train = train.scalar_coupling_constant.values.reshape(-1, 1)
x_test = pred_test.scalar_coupling_constant.values.reshape(-1, 1)

In [78]:
print (x_train[0:5])
print (y_train[0:5])
print (x_test[0:5])

[[ 84.8076]
 [-11.257 ]
 [-11.2548]
 [-11.2543]
 [ 84.8074]]
[[ 84.8076]
 [-11.257 ]
 [-11.2548]
 [-11.2543]
 [ 84.8074]]
[[  9.85089636]
 [177.44711685]
 [  2.04497147]
 [175.8211441 ]
 [ 10.29709315]]


In [83]:
# REMEMBER y_train are the real ones!
print (mean_absolute_error(y_train, x_train))
print (np.log(mean_absolute_error(y_train, x_train)))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV

# ElasticNet

In [ ]:
elastic = ElasticNet(alpha = 0.1)
elastic.fit(x_train, y_train)

In [ ]:
elastic_score = elastic.score(y_train, x_train)
elastic_score

In [ ]:
mae_elastic = mean_absolute_error(y_train, elastic.predict(x_train))
print (mae_elastic)
print (np.log(mae_elastic))

In [ ]:
y_elastic = elastic.predict(x_test)
print (x_test[0:5])
print (y_elastic[0:5])

# LASSO

In [ ]:
lasso = linear_model.Lasso(alpha=0.1)
lasso.fit(x_train, y_train)

In [ ]:
mae_lasso = mean_absolute_error(y_train, lasso.predict(x_train))
print (mae_lasso)
print (np.log(mae_lasso))

In [ ]:
lasso_score = lasso.score(x_train, y_train)
lasso_score

In [ ]:
y_lasso = lasso.predict(x_test)
print (x_test[0:5])
print (y_lasso[0:5])

# Submission

In [ ]:
lasso_score > elastic_score

In [ ]:
mae_lasso < mae_elastic

In [ ]:
if mae_lasso < mae_elastic:
    print ('Use Lasso')
    sub['scalar_coupling_constant'] = y_lasso
    sub.to_csv('lasso_postprocessing.csv', index=False)
    
else:
    print ('Use ElasticNet')
    sub['scalar_coupling_constant'] = y_elastic
    sub.to_csv('elastic_postprocessing.csv', index=False)   
    

In [ ]:
sub.head()

In [ ]:
pred_test.head()